In [2]:
import fiftyone as fo
from PIL import Image
import os

In [15]:
def crop_and_save_dataset(
    dataset, output_dir="cropped_dataset", target_size=(224, 224)
):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cropped_samples = []
    for split in ["train", "valid", "test"]:
        ds = dataset.match_tags(split)
        for sample in ds:
            image_path = sample.filepath
            image = Image.open(image_path)
            width, height = image.size

            for detection in sample.ground_truth.detections:
                label = detection.label
                bounding_box = (
                    detection.bounding_box
                )  # [top-left-x, top-left-y, width, height]

                # Convert relative bounding box coordinates to absolute pixel values
                x_min = int(bounding_box[0] * width)
                y_min = int(bounding_box[1] * height)
                x_max = int((bounding_box[0] + bounding_box[2]) * width)
                y_max = int((bounding_box[1] + bounding_box[3]) * height)

                # Crop the image using bounding box coordinates
                cropped_image = image.crop((x_min, y_min, x_max, y_max))

                # Resize the cropped image to the target size (224x224)
                resized_image = cropped_image.resize(
                    target_size, Image.LANCZOS
                )

                # Save the cropped and resized image
                cropped_image_name = f"{sample.id}_{label}.jpg"
                cropped_image_path = os.path.join(
                    output_dir, cropped_image_name
                )
                resized_image.save(cropped_image_path)

                # Create a new FiftyOne sample for the cropped image
                cropped_sample = fo.Sample(filepath=cropped_image_path)
                cropped_sample["label"] = fo.Classification(label=label)
                cropped_sample["tags"] = [split]  # Adding the tag as a list
                cropped_samples.append(cropped_sample)

    # Create a new FiftyOne dataset with the cropped images
    cropped_dataset_name = "cropped_images_with_splits"
    cropped_dataset = fo.Dataset(name=cropped_dataset_name)
    cropped_dataset.add_samples(cropped_samples)

    return cropped_dataset

In [ ]:
dataset = fo.Dataset.from_dir(
    dataset_dir="original_ds",
    dataset_type=fo.types.FiftyOneDataset,
)

In [ ]:
fo.Session(dataset, port=23671)

In [ ]:
cropped_dataset = crop_and_save_dataset(dataset)

In [ ]:
cropped_dataset.export(
    "cropped_classification_dataset", fo.types.FiftyOneDataset
)

In [ ]:
fo.Session(cropped_dataset, port=23672)

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [ ]:
class FiftyOneDataset(Dataset):
    def __init__(self, dataset, transform=None):
        """
        Args:
            dataset (fo.Dataset): FiftyOne dataset object.
            transform (callable, optional): A function/transform to apply to the images.
        """
        self.dataset = dataset
        self.transform = transform
        # Create a list of sample IDs
        self.sample_ids = [sample.id for sample in dataset]

    def __len__(self):
        return len(self.sample_ids)

    def __getitem__(self, idx):
        # Use sample ID to access the sample
        sample_id = self.sample_ids[idx]
        sample = self.dataset[sample_id]

        # Extracting the image file path and label from the sample
        image_path = sample.filepath
        label = sample["label"]  # Extract the label from the sample

        # Open the image file and convert to RGB
        image = Image.open(image_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, label


# Load your FiftyOne dataset
dt = fo.Dataset.from_dir(
    "cropped_classification_dataset", dataset_type=fo.types.FiftyOneDataset
)
view = dt.match_tags("train")
# Define transformations (excluding resize as images are already 224x224)
transform = transforms.Compose(
    [
        transforms.ToTensor(),  # Convert PIL image to tensor
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Normalize with VGG16's mean and std
    ]
)

# Create the dataset and dataloader
custom_dataset = FiftyOneDataset(view, transform=transform)
dataloader = DataLoader(
    custom_dataset, batch_size=32, shuffle=True, num_workers=4
)